In [38]:
import pandas as pd
import glob
import os
import numpy as np
from featureranker.utils import *
from featureranker.plots import *
from featureranker.rankers import *
from tqdm.auto import tqdm

xlsx_path = './betting_odds/'
txt_path = './game_data/'
paths = [path for path in glob.glob(xlsx_path + '*.xlsx')]
paths_txt = [path_txt for path_txt in glob.glob(txt_path + '*.txt')]
for path in paths:
    xlsx_dataframes = pd.read_excel(path)
    xlsx_dataframes.columns = xlsx_dataframes.columns.str.replace('[^a-zA-Z0-9]', '')
    view_data(xlsx_dataframes)

for path_txt in paths_txt:
    txt_dataframes = pd.read_csv(path_txt)
    txt_dataframes.columns = txt_dataframes.columns.str.replace('[^a-zA-Z0-9]', '')
    view_data(txt_dataframes)

# If you want to view the file in Excel, uncomment the following line
# os.startfile(xlsx_files[0])


There are no NaN values in the dataset
The column 9th has 0.1% NaN values.
There are no NaN values in the dataset
There are no NaN values in the dataset
The column 9th has 0.1% NaN values.
There are no NaN values in the dataset
The column 9th has 0.1% NaN values.
The column Pitcher has 0.6% NaN values.
The column 9th has 0.1% NaN values.
The column Pitcher has 0.2% NaN values.
The column 8th has 0.1% NaN values.
The column 9th has 0.1% NaN values.
There are no NaN values in the dataset
There are no NaN values in the dataset
There are no NaN values in the dataset
The column Unnamed: 13 has 99.8% NaN values.
The column Unnamed: 14 has 100.0% NaN values.
The column Unnamed: 15 has 100.0% NaN values.
The column Unnamed: 85 has 100.0% NaN values.
The column Unnamed: 87 has 100.0% NaN values.
The column roset001 has 46.8% NaN values.
The column hollm001 has 4.9% NaN values.
The column Unnamed: 159 has 99.4% NaN values.
The column Unnamed: 13 has 100.0% NaN values.
The column Unnamed: 14 has 

In [39]:
# print(xlsx_dataframes.head)
print(txt_dataframes)

      20220407  0  Thu  SDN  NL    1  ARI NL.1  1.1  2  ...  \
0     20220407  0  Thu  CIN  NL    1  ATL   NL    1  6  ...   
1     20220407  0  Thu  MIL  NL    1  CHN   NL    1  4  ...   
2     20220407  0  Thu  PIT  NL    1  SLN   NL    1  0  ...   
3     20220407  0  Thu  NYN  NL    1  WAS   NL    1  5  ...   
4     20220407  0  Thu  HOU  AL    1  ANA   AL    1  3  ...   
...        ... ..  ...  ...  ..  ...  ...  ...  ... ..  ...   
2424  20221005  0  Wed  KCA  AL  162  CLE   AL  162  2  ...   
2425  20221005  0  Wed  PHI  NL  162  HOU   AL  162  2  ...   
2426  20221005  0  Wed  ANA  AL  162  OAK   AL  162  2  ...   
2427  20221005  0  Wed  DET  AL  162  SEA   AL  162  4  ...   
2428  20221005  0  Wed  NYA  AL  162  TEX   AL  162  2  ...   

            Seth Beer.1  10.1  ellid002       Drew Ellis  5.3  perdg001  \
0       Travis d'Arnaud     2  dicka001   Alex Dickerson   10  swand001   
1         Jason Heyward     8  wisdp001   Patrick Wisdom    5  hoern001   
2         Yadier M

In [21]:
def process_dataframes(xlsx_dataframes):
    merged_df = pd.concat(xlsx_dataframes)
    home_teams_df = merged_df[merged_df['VH'] == 'H']
    visiting_teams_df = merged_df[merged_df['VH'] == 'V']

    selected_columns_df = pd.DataFrame()
    for i in range(len(xlsx_dataframes)):
        year = 2010 + i
        temp_df = pd.DataFrame({
            'date': home_teams_df['Date'].apply(lambda x: str(year) + str(x).zfill(4)).reset_index(drop=True),
            'home_team': home_teams_df['Team'].reset_index(drop=True),
            'visiting_team': visiting_teams_df['Team'].reset_index(drop=True),
            'home_open': home_teams_df['Open'].reset_index(drop=True),
            'visiting_open': visiting_teams_df['Open'].reset_index(drop=True)
        })
        selected_columns_df = pd.concat([selected_columns_df, temp_df])

    return selected_columns_df

print(process_dataframes(xlsx_dataframes))


TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"

In [ ]:
def add_player_names(selected_columns_df, txt_dataframes):
    player_names_df = pd.DataFrame()
    for txt_df in txt_dataframes:
        txt_df['date'] = txt_df.iloc[:, 0].apply(lambda x: pd.to_datetime(x.split(',')[0], format='%Y%m%d', errors='coerce'))
        for i in range(len(selected_columns_df)):
            match_date = selected_columns_df.iloc[i]['date']
            match_home_team = selected_columns_df.iloc[i]['home_team']
            match_visiting_team = selected_columns_df.iloc[i]['visiting_team']
            matching_row = txt_df[(txt_df['date'] == match_date) & (txt_df.iloc[:, 6].values == match_home_team) & (txt_df.iloc[:, 3].values == match_visiting_team)]
            if not matching_row.empty:
                if matching_row.shape[0] > 1:
                    selected_columns_df.at[i, 'home_players'] = matching_row.iloc[0, 132:158].values
                    selected_columns_df.at[i, 'visiting_players'] = matching_row.iloc[0, 105:131].values
                else: 
                    selected_columns_df.at[i, 'home_players'] = matching_row.iloc[:, 132:158].values[0]
                    selected_columns_df.at[i, 'visiting_players'] = matching_row.iloc[:, 105:131].values[0]
    return selected_columns_df

final_df = add_player_names(process_dataframes(xlsx_dataframes), txt_dataframes)
print(final_df)









IndexError: single positional indexer is out-of-bounds

In [ ]:
print(selected_columns_df)

NameError: name 'selected_columns_df' is not defined

SyntaxError: invalid syntax (37427434.py, line 1)